In [16]:
# Lasp module

import sys
sys.path.append('../../..')

import lasp.io
import lasp.convert

# Other
import pandas
import pathlib
import matplotlib.pyplot
import yaml

# PATH = pathlib.Path('./5-MumfordShahFSR')
# if not(PATH.exists()):
#     PATH.mkdir()

In [17]:
import dataset

IMAGES_PATH = pathlib.Path('../0-Images')
GENERATION_PATH = pathlib.Path('./ClassicDatasetConvolve')

INPUT = GENERATION_PATH / 'blurred'
GROUND_TRUTH = GENERATION_PATH / 'ground_truth'

if not(GENERATION_PATH.exists()):
    GENERATION_PATH.mkdir()
if not(INPUT.exists()):
    INPUT.mkdir()
if not(GROUND_TRUTH.exists()):
    GROUND_TRUTH.mkdir()

IMG_GEN_PKL = GENERATION_PATH / 'df_imgs.pkl'

In [18]:
for img_path in IMAGES_PATH.glob('*'):
    img = lasp.io.read(img_path)
    print('{} : {}'.format(img_path.stem, img.shape))

Cameraman : (256, 256)
Baboon : (512, 512)
Boats : (576, 720)
Goldhill : (512, 512)
Peppers : (512, 512)


In [20]:
df_imgs = pandas.DataFrame(columns=[ 'original', 'blur', 'decimation', 'noise' ])

decimation = -1


for img in IMAGES_PATH.glob('*'):

    if img.stem in [ 'Baboon', 'Goldhill', 'Peppers' ]:

        # for snr_db in [ 10, 20, 30, 40, 50, 60 ]:
        for snr_db in range(10, 61, 5):

            noise = lasp.convert.snrdb_to_snr(snr_db)

            for blur in [ (7, 3), (9, 3), (15, 3), (21, 3), (45, 3) ]:

                df_imgs = dataset.add_image(df_imgs, img, blur, decimation, noise)

df_imgs

,original,blur,decimation,noise
0,../0-Images/Baboon.bmp,"(7, 3)",-1,3.162278
1,../0-Images/Baboon.bmp,"(9, 3)",-1,3.162278
2,../0-Images/Baboon.bmp,"(15, 3)",-1,3.162278
3,../0-Images/Baboon.bmp,"(21, 3)",-1,3.162278
4,../0-Images/Baboon.bmp,"(45, 3)",-1,3.162278
...,...,...,...,...
160,../0-Images/Peppers.bmp,"(7, 3)",-1,1000.0
161,../0-Images/Peppers.bmp,"(9, 3)",-1,1000.0
162,../0-Images/Peppers.bmp,"(15, 3)",-1,1000.0
163,../0-Images/Peppers.bmp,"(21, 3)",-1,1000.0


In [21]:
import scipy.signal
import lasp.filters.linear
import lasp.utils
import lasp.noise

def make_images(df_imgs: pandas.DataFrame, save_path: pathlib.Path) -> None:

    for index in df_imgs.index:

        # CURRENT = save_path / str(index)
        # if not(CURRENT.exists()):
        #     CURRENT.mkdir()

        
        img_datas = df_imgs.loc[index]

        img_path = img_datas['original']
        blur = img_datas['blur']
        decim = img_datas['decimation']
        noise = img_datas['noise']


        out = lasp.io.read(img_path) # x
        lasp.io.save(out, GROUND_TRUTH / '{}.npy'.format(index))
        #lasp.io.save(out, GROUND_TRUTH / '{}.png'.format(index))
        

        if (blur[0] > 0) and (blur[1] > 0):
            kernel = lasp.filters.linear.gaussian_filter(size=blur[0], sigma=blur[1]) # Hx
            out = scipy.signal.convolve2d(out, kernel, mode='same')
            
        if decim > 0:
            out = lasp.utils.decimation(out, decim) # SHx

        if noise >= 0:
            out = lasp.noise.awgn(out, noise) # SHx + n

        normalized = lasp.utils.normalize(out)
        lasp.io.save(normalized, INPUT / '{}.npy'.format(index))
        #lasp.io.save(normalized, INPUT / '{}.png'.format(index))

In [22]:
make_images(df_imgs, GENERATION_PATH)
pandas.to_pickle(df_imgs, IMG_GEN_PKL)